In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from tqdm import tqdm
import torch.nn.functional as F
from gensim.models import Word2Vec
from sklearn import metrics
from sklearn.metrics import classification_report
import itertools
import random

In [10]:
import torch
import torch.nn as nn

In [11]:
device = 'cuda'

In [12]:
pos_weight = 881/323

In [13]:
class BertModel(nn.Module):
    def __init__(self, voc_size:int=30000, seq_len: int=512, d_model: int=384, d_ff:int=3072, pad_idx: int=1,
                num_encoder: int=12, num_heads: int=12, n_layers=6, dropout: float=0.1):
        super(BertModel, self).__init__()
        self.pad_idx = pad_idx
        self.layers = nn.ModuleList([nn.TransformerEncoderLayer(d_model=d_model, nhead=8, batch_first=True) for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.classifier = nn.Linear(d_model, 1)
        
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        output = input
        for layer in self.layers:
            output = layer(output)
        #output = output.mean(dim=1, keepdim=True)
        output = output[:,-1]
        h_pooled = self.activ1(self.fc(output)) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2]
        return logits_clsf # [B, S, D_model]

In [14]:
def log_metrics(preds, labels):
    preds = torch.stack(preds)
    preds = preds.cpu().detach().numpy()
    labels = torch.stack(labels)
    labels = labels.cpu().detach().numpy()
    
    
    ## Method 2 using ravel()
    fpr_micro, tpr_micro, _ = metrics.roc_curve(labels.ravel(), preds.ravel())
    auc_micro = metrics.auc(fpr_micro, tpr_micro)
    n=10
    preds = [round(i) for i in preds.ravel()]
    acc = metrics.accuracy_score(labels.ravel(), preds)
    f1 = metrics.f1_score(labels.ravel(), preds)
    preds = [preds[i*n:(i+1)*n] for i in range(len(preds)//6)]
    #target_names = list(genre_list.keys())
    #print(classification_report(labels,preds, target_names=target_names))
    return {"auc_micro": auc_micro, "acc" : acc, "f1" : f1, 'pred' : preds}

In [15]:
def train_one_epoch(dataloader, model, optimizer, device, loss_fn):
    model.train()
    tk0 = tqdm(dataloader, total=len(dataloader))
    total_loss = 0.0
    
    for source_tensor, label in tk0:
        optimizer.zero_grad()
        output = model(source_tensor.to(device))
        loss = loss_fn(output.view(-1, output.size(-1)), torch.FloatTensor([label]).to(device))
        
        total_loss += loss
        loss.backward()
        optimizer.step()
        
    avg_train_loss = total_loss / len(dataloader) 
    print(" Average training loss: {0:.2f}".format(avg_train_loss))  

def test_one_epoch(testset, model, loss_fn):
    model.eval()
    tk0 = tqdm(testset, total=len(testset))
    targets = []
    outputs = []
    total_loss = 0.0
    with torch.no_grad():
        for source_tensor, label in tk0:
            label = torch.FloatTensor([label])
            output = model(source_tensor.to(device))
            loss = loss_fn(output.view(-1, output.size(-1)), label.to(device))
            total_loss += loss
            targets.extend(label)
            outputs.extend(torch.sigmoid(output[0]).to('cpu'))
    avg_valid_loss = total_loss / len(testset) 
    score = log_metrics(outputs, targets)
    print(" Average valid loss: {0:.2f}".format(avg_valid_loss))  
    print('AUC_SCORE: ', score["auc_micro"], " acc: ", score["acc"], "f1: ", score["f1"])
    return score, avg_valid_loss
    
def fit(model, train_dataloader, valid_dataloader=None,test_dataloader=None, EPOCHS=3, lr=0.0000005):
    loss_fn = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight)) #ignore padding
    optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
    for i in range(EPOCHS):
        print(f"EPOCHS:{i+1}")
        print('TRAIN')
        train_one_epoch(train_dataloader, model, optimizer, device, loss_fn)
        auc, loss = test_one_epoch(valid_dataloader,model, loss_fn)
        if i == 0:
            torch.save(model,'rnn_best.model')
            best_loss = loss
        if  loss<best_loss:
            best_loss = loss
            print(i,' EPOCH BEST MODEL!')
            torch.save(model,'rnn_best.model')
        print('Test')
        auc, loss = test_one_epoch(test_dataloader,model, loss_fn)
    best_model = torch.load('rnn_best.model')
    auc, loss  = test_one_epoch(test_dataloader,best_model, loss_fn)
    print('BEST MODEL')
    print('AUC_SCORE: ', auc["auc_micro"], " acc: ", auc["acc"], "f1: ", auc["f1"])
    return auc

            best_f1 = auc['f1']
        if  auc['f1']>best_f1:
            best_f1 = auc['f1']

In [16]:
import pickle
with open('movie_split_flashback.pickle','rb') as fw:
    movie_split = pickle.load(fw)

In [17]:
path1 = 'word2vec_model_edge/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
end = torch.ones(1,1,400)
for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'flashback'==keyword:
                    cult = 1
                    
                    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])

/tmp/ipykernel_16566/544602680.py:40: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/torch/csrc/utils/tensor_numpy.cpp:172.)
  emb = torch.from_numpy(emb)


In [18]:
bert = BertModel(d_model=400).to('cuda')
score = fit(bert, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:09<00:00, 102.94it/s]


 Average training loss: 1.02


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 863.73it/s]


 Average valid loss: 1.00
AUC_SCORE:  0.5852272727272727  acc:  0.5916666666666667 f1:  0.3098591549295775
Test


...

 Average valid loss: 1.10
AUC_SCORE:  0.536221590909091  acc:  0.6416666666666667 f1:  0.3174603174603175
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 853.41it/s]


 Average valid loss: 1.08
AUC_SCORE:  0.5396769662921349  acc:  0.6115702479338843 f1:  0.3380281690140845


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 854.73it/s]

 Average valid loss: 1.02
AUC_SCORE:  0.47296348314606745  acc:  0.5785123966942148 f1:  0.28169014084507044
BEST MODEL
AUC_SCORE:  0.47296348314606745  acc:  0.5785123966942148 f1:  0.28169014084507044
{'auc_micro': 0.47296348314606745, 'acc': 0.5785123966942148, 'f1': 0.28169014084507044, 'pred': [[0, 1, 1, 1, 1, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 0, 0], [0, 1, 1, 1, 0, 0, 0, 1, 0, 1], [0, 0, 0, 1, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 1, 1], [1, 1, 1, 1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1, 0, 1, 0, 1], [0, 0, 1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 1, 0, 1, 0, 0, 1, 1], [0], [], [], [], [], [], [], []]}


In [104]:
random.randint(0,100)

19

In [19]:
import random

# Doc2vce

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# SUM

In [20]:
import pickle
with open('emb_sum.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [21]:
path1 = 'word2vec_model_char/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
end = torch.ones(1,1,384)

for filename in filenames:
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'flashback'==keyword:
                    cult = 1
    if movie_name not in movie_emb.keys():
        continue
        
    embs = torch.zeros(1,len(movie_emb[movie_name])+1,384)
    for i,emb in enumerate(movie_emb[movie_name]):
        emb = torch.from_numpy(np.nan_to_num(emb))
        embs[0][i] = emb
        
    embs[0][-1] = end
        
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
        #valid_data.append([embs, [cult]])

In [22]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

953
120
119


In [23]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 953/953 [00:07<00:00, 119.52it/s]


 Average training loss: 1.02


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 875.52it/s]


 Average valid loss: 1.01
AUC_SCORE:  0.46626420454545453  acc:  0.6416666666666667 f1:  0.0851063829787234
Test


...

 Average valid loss: 1.06
AUC_SCORE:  0.5291193181818182  acc:  0.6083333333333333 f1:  0.2769230769230769
Test


100%|████████████████████████████████████████| 119/119 [00:00<00:00, 869.28it/s]


 Average valid loss: 1.06
AUC_SCORE:  0.5153558052434457  acc:  0.6134453781512605 f1:  0.28125


100%|████████████████████████████████████████| 119/119 [00:00<00:00, 868.07it/s]

 Average valid loss: 1.01
AUC_SCORE:  0.5176029962546816  acc:  0.5882352941176471 f1:  0.30985915492957744
BEST MODEL
AUC_SCORE:  0.5176029962546816  acc:  0.5882352941176471 f1:  0.30985915492957744
{'auc_micro': 0.5176029962546816, 'acc': 0.5882352941176471, 'f1': 0.30985915492957744, 'pred': [[0, 1, 0, 1, 1, 0, 0, 1, 0, 0], [1, 1, 0, 1, 1, 1, 0, 1, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 1, 1, 0, 0, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 1, 0, 0], [0, 1, 1, 0, 1, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 1, 1], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 1, 0, 0, 1, 0], [0, 1, 1, 0, 0, 0, 0, 1, 0], [], [], [], [], [], [], []]}


# Doc2vec

In [24]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [25]:
path1 = 'doc2vec/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'flashback'==keyword:
                    cult = 1
        
    w2v=Doc2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.dv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.dv.index_to_key:
            emb = w2v.dv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [26]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 119.01it/s]


 Average training loss: 1.02


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 919.41it/s]

 Average valid loss: 1.01
AUC_SCORE:  0.5387073863636364  acc:  0.5833333333333334 f1:  0.3055555555555556


...

 Average valid loss: 1.02
AUC_SCORE:  0.5944602272727273  acc:  0.625 f1:  0.3283582089552239
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 940.91it/s]


 Average valid loss: 1.17
AUC_SCORE:  0.43433988764044945  acc:  0.5371900826446281 f1:  0.09677419354838711


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 937.15it/s]

 Average valid loss: 1.07
AUC_SCORE:  0.4224016853932584  acc:  0.5454545454545454 f1:  0.1791044776119403
BEST MODEL
AUC_SCORE:  0.4224016853932584  acc:  0.5454545454545454 f1:  0.1791044776119403
{'auc_micro': 0.4224016853932584, 'acc': 0.5454545454545454, 'f1': 0.1791044776119403, 'pred': [[0, 1, 1, 0, 1, 1, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 1, 0, 1, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 1, 0, 1, 0, 0, 0], [1, 0, 0, 1, 0, 0, 1, 0, 0, 1], [1, 0, 1, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 1, 1, 1, 0], [0], [], [], [], [], [], [], []]}


# 평균 임베딩

In [27]:
import pickle
with open('emb_avg2.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [28]:
path1 = 'word2vec_model_char/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
end = torch.ones(1,1,384)

for filename in filenames:
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'flashback'==keyword:
                    cult = 1
        
    embs = torch.zeros(1,len(movie_emb[movie_name])+1,384)
    for i,emb in enumerate(movie_emb[movie_name]):
        emb = torch.from_numpy(np.nan_to_num(emb))
        embs[0][i] = emb
        
    embs[0][-1] = end
        
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
        #valid_data.append([embs, [cult]])

In [29]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

963
120
121


In [30]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:08<00:00, 119.86it/s]


 Average training loss: 1.02


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 850.01it/s]

 Average valid loss: 1.02
AUC_SCORE:  0.3895596590909091  acc:  0.5166666666666667 f1:  0.17142857142857143


...

 Average valid loss: 1.18
AUC_SCORE:  0.4719460227272727  acc:  0.5833333333333334 f1:  0.2647058823529412
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 846.45it/s]


 Average valid loss: 1.05
AUC_SCORE:  0.5737359550561798  acc:  0.6942148760330579 f1:  0.3508771929824561


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 843.43it/s]

 Average valid loss: 1.01
AUC_SCORE:  0.5410814606741573  acc:  0.6115702479338843 f1:  0.25396825396825395
BEST MODEL
AUC_SCORE:  0.5410814606741573  acc:  0.6115702479338843 f1:  0.25396825396825395
{'auc_micro': 0.5410814606741573, 'acc': 0.6115702479338843, 'f1': 0.25396825396825395, 'pred': [[1, 0, 0, 0, 0, 0, 1, 1, 0, 0], [1, 0, 0, 1, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 0, 0], [1, 0, 1, 0, 0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 1, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 0, 1, 1, 0, 1, 0], [0], [], [], [], [], [], [], []]}


# 그냥 metapath

In [31]:
path1 = 'word2vec_model_char/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'flashback'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [32]:
print(len(train_data))
print(len(valid_data))
print(len(test_data))

963
120
121


In [33]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:07<00:00, 121.65it/s]


 Average training loss: 1.02


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 883.40it/s]


 Average valid loss: 1.01
AUC_SCORE:  0.4897017045454545  acc:  0.6833333333333333 f1:  0.2962962962962963
Test


...

 Average valid loss: 1.13
AUC_SCORE:  0.5028409090909092  acc:  0.6083333333333333 f1:  0.25396825396825395
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 876.02it/s]


 Average valid loss: 1.06
AUC_SCORE:  0.5400280898876405  acc:  0.5785123966942148 f1:  0.31999999999999995


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 875.26it/s]

 Average valid loss: 1.02
AUC_SCORE:  0.4483848314606742  acc:  0.5619834710743802 f1:  0.2318840579710145
BEST MODEL
AUC_SCORE:  0.4483848314606742  acc:  0.5619834710743802 f1:  0.2318840579710145
{'auc_micro': 0.4483848314606742, 'acc': 0.5619834710743802, 'f1': 0.2318840579710145, 'pred': [[0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 0, 1, 0, 1, 1, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 1, 0, 1], [1, 0, 0, 0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0, 1, 1], [0, 1, 0, 0, 0, 0, 0, 1, 0, 0], [1, 1, 0, 0, 0, 1, 0, 1, 0, 0], [0, 1, 1, 1, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 1, 0, 1, 0, 1], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 1, 1, 0, 0], [0], [], [], [], [], [], [], []]}


# Main_act

In [34]:
path1 = 'word2vec_model_extra+emb/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'flashback'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [35]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 963/963 [00:07<00:00, 120.48it/s]


 Average training loss: 1.02


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 878.30it/s]

 Average valid loss: 1.02
AUC_SCORE:  0.46768465909090906  acc:  0.6083333333333333 f1:  0.2769230769230769


...

 Average valid loss: 1.09
AUC_SCORE:  0.5411931818181819  acc:  0.6416666666666667 f1:  0.37681159420289856
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 881.43it/s]


 Average valid loss: 1.05
AUC_SCORE:  0.559691011235955  acc:  0.6446280991735537 f1:  0.3582089552238806


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 881.94it/s]

 Average valid loss: 1.01
AUC_SCORE:  0.5547752808988764  acc:  0.6115702479338843 f1:  0.3380281690140845
BEST MODEL
AUC_SCORE:  0.5547752808988764  acc:  0.6115702479338843 f1:  0.3380281690140845
{'auc_micro': 0.5547752808988764, 'acc': 0.6115702479338843, 'f1': 0.3380281690140845, 'pred': [[0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 1, 0, 1], [1, 0, 1, 0, 1, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [1, 1, 0, 0, 1, 0, 1, 1, 0, 0], [0, 1, 1, 1, 0, 1, 0, 1, 1, 0], [1, 0, 1, 1, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 0, 0], [0], [], [], [], [], [], [], []]}


# pos

In [36]:
path1 = 'word2vec_model_extra+pos/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,400)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'flashback'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [37]:
bert2 = BertModel(d_model=400).to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=25)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 962/962 [00:07<00:00, 121.44it/s]


 Average training loss: 1.03


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 860.99it/s]


 Average valid loss: 1.02
AUC_SCORE:  0.4215198863636364  acc:  0.65 f1:  0.08695652173913043
Test


...

 Average valid loss: 1.09
AUC_SCORE:  0.48579545454545453  acc:  0.6083333333333333 f1:  0.25396825396825395
Test


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 849.87it/s]


 Average valid loss: 1.02
AUC_SCORE:  0.5610795454545455  acc:  0.6166666666666667 f1:  0.28125


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 849.38it/s]

 Average valid loss: 1.02
AUC_SCORE:  0.5067471590909091  acc:  0.6166666666666667 f1:  0.30303030303030304
BEST MODEL
AUC_SCORE:  0.5067471590909091  acc:  0.6166666666666667 f1:  0.30303030303030304
{'auc_micro': 0.5067471590909091, 'acc': 0.6166666666666667, 'f1': 0.30303030303030304, 'pred': [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 1, 1, 0, 0, 0, 1, 1], [0, 1, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1], [1, 0, 0, 0, 1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 1, 0, 1], [0, 0, 0, 1, 1, 0, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [], [], [], [], [], [], [], []]}


# text_rank

In [38]:
path1 = 'word2vec_model_text/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'flashback'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [39]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 960/960 [00:08<00:00, 118.61it/s]


 Average training loss: 1.01


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 869.67it/s]


 Average valid loss: 1.01
AUC_SCORE:  0.5042613636363636  acc:  0.6583333333333333 f1:  0.16326530612244897
Test


...

 Average valid loss: 1.12
AUC_SCORE:  0.5184659090909091  acc:  0.65 f1:  0.30000000000000004
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 849.37it/s]


 Average valid loss: 1.04
AUC_SCORE:  0.5751404494382022  acc:  0.6776859504132231 f1:  0.38095238095238093


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 850.31it/s]

 Average valid loss: 1.02
AUC_SCORE:  0.5112359550561797  acc:  0.5619834710743802 f1:  0.273972602739726
BEST MODEL
AUC_SCORE:  0.5112359550561797  acc:  0.5619834710743802 f1:  0.273972602739726
{'auc_micro': 0.5112359550561797, 'acc': 0.5619834710743802, 'f1': 0.273972602739726, 'pred': [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 1, 1, 1, 0, 1], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0, 1], [1, 0, 1, 0, 1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [1, 0, 1, 0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 1, 1, 1, 1, 0, 0], [0, 1, 1, 1, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 1, 1, 1, 1, 0, 0], [0, 0, 1, 0, 0, 0, 1, 0, 1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0], [], [], [], [], [], [], []]}


In [40]:
path1 = 'word2vec_model_text2/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'flashback'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [41]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 960/960 [00:08<00:00, 119.00it/s]


 Average training loss: 1.02


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 838.87it/s]

 Average valid loss: 1.02
AUC_SCORE:  0.4147727272727273  acc:  0.5833333333333334 f1:  0.13793103448275862


...

 Average valid loss: 1.21
AUC_SCORE:  0.4616477272727273  acc:  0.575 f1:  0.3013698630136986
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 893.98it/s]


 Average valid loss: 1.01
AUC_SCORE:  0.5976123595505618  acc:  0.6198347107438017 f1:  0.3235294117647059


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 886.83it/s]

 Average valid loss: 1.01
AUC_SCORE:  0.5063202247191011  acc:  0.5537190082644629 f1:  0.2894736842105263
BEST MODEL
AUC_SCORE:  0.5063202247191011  acc:  0.5537190082644629 f1:  0.2894736842105263
{'auc_micro': 0.5063202247191011, 'acc': 0.5537190082644629, 'f1': 0.2894736842105263, 'pred': [[0, 1, 1, 0, 1, 0, 1, 0, 0, 0], [0, 1, 1, 0, 1, 1, 0, 1, 1, 1], [0, 0, 0, 1, 0, 0, 0, 1, 1, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 1, 0, 1], [1, 1, 0, 0, 0, 1, 0, 1, 1, 0], [0, 0, 0, 0, 1, 0, 0, 1, 0, 1], [1, 0, 1, 0, 0, 1, 1, 1, 0, 0], [0, 0, 0, 1, 0, 1, 0, 1, 0, 0], [0, 0, 0, 1, 1, 1, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0, 1, 0, 0], [0], [], [], [], [], [], [], []]}


In [42]:
path1 = 'word2vec_model_text3/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'flashback'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num+1,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    embs[0][-1] = end
        
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [43]:
bert2 = BertModel().to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=40)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 960/960 [00:07<00:00, 122.25it/s]


 Average training loss: 1.03


100%|████████████████████████████████████████| 120/120 [00:00<00:00, 900.08it/s]


 Average valid loss: 1.01
AUC_SCORE:  0.5017755681818181  acc:  0.6083333333333333 f1:  0.29850746268656714
Test


...

 Average valid loss: 1.10
AUC_SCORE:  0.5419034090909091  acc:  0.6083333333333333 f1:  0.3561643835616438
Test


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 878.59it/s]


 Average valid loss: 1.05
AUC_SCORE:  0.547752808988764  acc:  0.5950413223140496 f1:  0.3466666666666667


100%|████████████████████████████████████████| 121/121 [00:00<00:00, 873.79it/s]

 Average valid loss: 1.02
AUC_SCORE:  0.5042134831460674  acc:  0.5785123966942148 f1:  0.21538461538461537
BEST MODEL
AUC_SCORE:  0.5042134831460674  acc:  0.5785123966942148 f1:  0.21538461538461537
{'auc_micro': 0.5042134831460674, 'acc': 0.5785123966942148, 'f1': 0.21538461538461537, 'pred': [[0, 0, 0, 1, 0, 0, 0, 1, 1, 0], [0, 0, 0, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 1, 0, 1, 0, 1], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 0, 1, 1, 0, 0], [0, 0, 1, 1, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0], [], [], [], [], [], [], []]}


# 거리로 분석

In [156]:
path1 = 'word2vec_model_dim=384/'
path2 = 'data/scriptbase/'
filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
one = 0
zero = 0
end = torch.ones(1,1,384)

for filename in filenames:
    genres = []
    movie_name = filename[:-14]
    cult = 0
    with open(path2+movie_name+'/processed/imdb_meta.txt','r') as f:
        lines = f.readlines()
        for line in lines:
            if 'keyword' in line:
                try:
                    keyword  = line.strip().split('\t')[1]
                    keywords.append(keyword)
                except:
                    pass
                if 'surprise ending'==keyword:
                    cult = 1
        
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1

    ans = torch.zeros(1,1,512)
    for i in range(scene_num):
        if i ==0:
            continue
        #ans[0,0,i] += util.cos_sim(embs[0][i],embs[0][i-1])[0][0]
        ans[0,0,i] += torch.cdist(embs[0][i].unsqueeze(0),embs[0][i-1].unsqueeze(0))[0][0]
        
    embs = ans
    
        
    rand = random.randint(0,100)    
    if movie_name in movie_split['train']:
        train_data.append([embs, [cult]])
    elif movie_name in movie_split['valid']:
        valid_data.append([embs, [cult]])
    
    elif movie_name in movie_split['test']:
        test_data.append([embs, [cult]])
    
    if cult:
        one += 1
    else:
        zero += 1

In [157]:
bert2 = BertModel(d_model=512).to('cuda')
score = fit(bert2, train_data,valid_data,test_data,EPOCHS=20)
print(score)

EPOCHS:1
TRAIN


100%|████████████████████████████████████████| 929/929 [00:07<00:00, 127.24it/s]


 Average training loss: 1.07


100%|████████████████████████████████████████| 117/117 [00:00<00:00, 980.24it/s]


 Average valid loss: 1.06
AUC_SCORE:  0.5272873194221509  acc:  0.4700854700854701 f1:  0.3404255319148936
EPOCHS:2
TRAIN


...

 Average training loss: 1.02


100%|████████████████████████████████████████| 117/117 [00:00<00:00, 989.84it/s]


 Average valid loss: 1.13
AUC_SCORE:  0.4562600321027287  acc:  0.47863247863247865 f1:  0.2823529411764706


100%|████████████████████████████████████████| 117/117 [00:00<00:00, 985.66it/s]

 Average valid loss: 1.06
AUC_SCORE:  0.5497592295345104  acc:  0.41025641025641024 f1:  0.3551401869158879
BEST MODEL
AUC_SCORE:  0.5497592295345104  acc:  0.41025641025641024 f1:  0.3551401869158879
{'auc_micro': 0.5497592295345104, 'acc': 0.41025641025641024, 'f1': 0.3551401869158879, 'pred': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 1, 0, 1, 1, 0, 0, 0, 1, 1], [0, 1, 1, 0, 1, 1, 1, 0, 1, 0], [1, 1, 1, 1, 1, 1, 0, 0, 1, 0], [0, 1, 0, 1, 1, 1, 1, 1, 1, 1], [1, 1, 0, 1, 0, 1, 1, 1, 0, 1], [1, 1, 1, 0, 1, 1, 0, 0, 1, 1], [0, 1, 1, 1, 1, 1, 0, 1, 1, 0], [1, 0, 1, 0, 1, 1, 0, 1, 0, 1], [1, 1, 1, 0, 1, 0, 0, 0, 0, 1], [1, 1, 1, 1, 0, 0, 1, 1, 0, 1], [1, 0, 1, 0, 1, 1, 0], [], [], [], [], [], [], []]}
